In [2]:
import numpy as np
import pandas as pd

import pytwits
import requests

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
ticker_list = ["GOOG", "AMZN", "FB", "MSFT", "ADBE", "BABA", "BIDU", "BKNG", "BZUN", "MA", "MTCH", "MOMO", "NFLX", "PCTY", 
               "PYPL", "RP", "CRM", "SINA", "SQ", "ZG", "CTRP", "NVDA", "RNG", "WB", "ENV", "WUBA", "GDDY"]
tweets = []
tickers = []
access_token = '26c8e5b3d7bd2903f6091364ae85a91cce9b9aa4'
stocktwits = pytwits.StockTwits(access_token=access_token) # This would also work without passing token.

for ticker in ticker_list:
    symbol, cursor, messages = stocktwits.streams(path = 'symbol', id = ticker) # set limit on messages by limit = __, max = 30)
    for msg in messages:
        tweets.append(msg.body)
        tickers.append(symbol.symbol)    

tweet_df = pd.DataFrame({'Ticker': tickers, 'Tweet': tweets})

In [4]:
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("&#39;", "'")
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("$", "")
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace('&quot;', '"')
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("&amp;", "&")

In [5]:
def sentiment(tweet):
    analyze = SentimentIntensityAnalyzer()
    comp = analyze.polarity_scores(tweet)['compound']
    if comp > 0:
        return "Positive"
    elif comp < 0:
        return "Negative"
    else:
        return "Neutral"

def polarity(tweet):
    analyze = SentimentIntensityAnalyzer()
    return analyze.polarity_scores(tweet)['compound']

In [6]:
tweet_df['Sentiment'] = tweet_df['Tweet'].apply(lambda x: sentiment(x))
tweet_df['Score'] = tweet_df['Tweet'].apply(lambda x: polarity(x))
tweet_df.iloc[:10]

,Ticker,Tweet,Sentiment,Score
0,GOOG,SPY If Mrkt keeps the good news about China th...,Positive,0.1779
1,GOOG,Will GOOG WMT AMZN NFLX monies Now Rotate into...,Neutral,0.0000
2,GOOG,#Update(7) GOOG Sep-21 1195 Puts Up +87%. si...,Neutral,0.0000
3,GOOG,GOOGL goog amzn ba New round of china tariff t...,Negative,-0.2732
4,GOOG,"ES_F SPY SPX jump in at any price, bulls are l...",Positive,0.1779
5,GOOG,GOOGL goog amzn shorts gonna get killed. Cover...,Negative,-0.6705
6,GOOG,goog EXP:9/14/2018|MaxPain:1180.0|HighPutOI:11...,Neutral,0.0000
7,GOOG,"Pinging data across a 1,000-kilometer span, GO...",Neutral,0.0000
8,GOOG,GOOG \nAlphabet tries to vent off oversold s...,Neutral,0.0000
9,GOOG,"Google, Facebook Dealt Blow by EU Lawmakers in...",Negative,-0.3818


In [7]:
tweet_summary = tweet_df.groupby(['Ticker', 'Sentiment'], as_index=False).agg({'Tweet':'count','Score':'mean'}).rename(columns = {
    'Tweet':'Tweet_count', 'Score':'Avg_Score'})
tweet_summary.iloc[:10]

,Ticker,Sentiment,Tweet_count,Avg_Score
0,ADBE,Negative,4,-0.306600
1,ADBE,Neutral,19,0.000000
2,ADBE,Positive,7,0.379443
3,AMZN,Negative,6,-0.334317
4,AMZN,Neutral,17,0.000000
5,AMZN,Positive,7,0.451729
6,BABA,Negative,4,-0.382400
7,BABA,Neutral,17,0.000000
8,BABA,Positive,9,0.485200
9,BIDU,Negative,8,-0.457612
